In [ ]:
import skimage
import skimage.io as imio
import skimage.transform as trans
import os
from collections import Counter
import math
import numpy as np
from copy import deepcopy
from scipy.fftpack import dct, idct

import skimage
import skimage.io as imio
import skimage.transform as trans
import matplotlib as mpl
import matplotlib.pyplot as plt

VALIDATION = True

#default block size for training and testing
height = 128
width = 128

def block_training_data(block_height=height,block_width=width):
    # read raw image data 
    rawpath = './training_raw/'
    raw_name_list = os.listdir(rawpath)
    raw_name_list.sort()
    rawarray = []
    for name in raw_name_list:
        if '.raw' in name:
            rawarray.append(name)
    # extract data from green sensor 
    global green1,green2
    green1 = []
    green2 = []
    green = []
    
    num = len(rawarray)
    #print('length of rawarray is: ' + str(num))
    for k in range(0,num):
        pop = np.fromfile(rawpath+rawarray[k],dtype=np.dtype('int16'))
        pop = pop[28672 : ]
        pop = np.reshape(pop,(2160,3840))
#         a = pop[112:2160,0:2048]
#         b = pop[111:2159,2048:3840]
        a = pop[1:2049,0:2048]
        b = pop[0:2048,2048:3840]
        c = np.concatenate((b,a),axis=1)
        for i in range(0,8):
            for j in range(0,15):
                add_green1 = c[block_height*2*i:block_height*2*(i+1):2,block_width*2*j:block_width*2*(j+1):2]
                add_green2 = c[block_height*2*i+1:block_height*2*(i+1)+1:2,block_width*2*j+1:block_width*2*(j+1)+1:2]
                green1.append(add_green1)
                green2.append(add_green2)
        add_green = c[::2,::2]
        green.append(add_green)
        
    green = np.array(np.array(green)/64,dtype='uint8')
    print(type(green))
    print(green.shape)
    #save =  np.reshape(green,(green.shape[0],green.shape[1],green.shape[2],1))
    #np.save('./preprocess/unblocked.npy',save)
    green1 = np.array(np.array(green1)/64,dtype='uint8')
    green2 = np.array(np.array(green2)/64,dtype='uint8')
    
    for k in range(0,num):
        implot = plt.imshow(green[k,:,:])
        plt.title('raw_training block_id = ' + str(k))
        plt.colorbar()
        plt.show()
        plt.imsave(rawpath+ str(k)+'.png', green[k,:,:])
    
    return green1

def block_test_data(block_height=height,block_width=width):

    testpath = './test_raw/'
    test_name_list = os.listdir(testpath)
    test_name_list.sort()
    testarray = []
    for name in test_name_list:
        if '.raw' in name:
            testarray.append(name)
    num = len(testarray)
    #print('length of testarray is: ' + str(num))
    for k in range(0,num):
        pop = np.fromfile(testpath+testarray[k],dtype=np.dtype('int16'))
        pop = pop[28672 : ]
        pop = np.reshape(pop,(2160,3840))
#         a = pop[112:2160,0:2048]
#         b = pop[111:2159,2048:3840]
        a = pop[1:2050,0:2048]
        b = pop[0:2049,2048:3840]
        c = np.concatenate((b,a),axis=1)        
        green1 = []
        green2 = []
        green = []
        for i in range(0,8):
            for j in range(0,15):
                add_green1 = c[block_height*2*i:block_height*2*(i+1):2,block_width*2*j:block_width*2*(j+1):2]
                add_green2 = c[block_height*2*i+1:block_height*2*(i+1)+1:2,block_width*2*j+1:block_width*2*(j+1)+1:2]
                green1.append(add_green1)
                green2.append(add_green2)
        green1 = np.array(np.array(green1)/64,dtype='uint8')
        green2 = np.array(np.array(green2)/64,dtype='uint8')
        add_green = c[::2,::2]
        green = add_green
        green = np.array(np.array(green)/64,dtype='uint8')
        
        save = np.reshape(green,(green.shape[0],green.shape[1],1))
        #np.save('./testset/test1'+str(k)+'.npy',green1)
        
        
        implot = plt.imshow(green)
        plt.title('raw_test block_id = ' + str(k))
        plt.colorbar()
        plt.show()
        plt.imsave(testpath+ str(k)+'.png', green)
        
        return green1

def dct_testtrain_data(blockset):
    dct_set = []
    num = blockset.shape[0]
    for block_id in range(num):
        frame = blockset[block_id,:,:]
        frame = dct2D(frame)
        dct_set.append(frame)
    dct_set = np.array(dct_set)
    return dct_set
        
    
def idct_testtrain_data(blockset):
    idct_set = []
    num = blockset.shape[0]
    for block_id in range(num):
        frame = blockset[block_id, :, :]
        frame = idct2D(frame)
        idct_set.append(frame)
    idct_set = np.array(dct_set)
    return idct_set
    
def dct2D(x):
	tmp = dct(x, type=2, norm='ortho').transpose()
	return dct(tmp, type=2 ,norm='ortho').transpose()

def idct2D(x):
	tmp = idct(x, type=2 ,norm='ortho').transpose()
	return idct(tmp, type=2 ,norm='ortho').transpose()

def get_dct_co(x):
    """
    takes original pixel values as input
    generate the compressed and uncompressed DCT coefficients
    :param x: np.array of shape (n, 128, 128)
    :return: x(compressed), y(uncompressed)
    """
    num = int(x.shape[0])
    b = int(x.shape[1] / 8)
    x = np.array(x, dtype='float32') - 128
    y = deepcopy(x)
    q_matrix = np.array([
        16, 11, 10, 16, 24, 40, 51, 61,
        12, 12, 14, 19, 26, 58, 60, 55,
        14, 13, 16, 24, 40, 57, 69, 56,
        14, 17, 22, 29, 51, 87, 80, 62,
        18, 22, 37, 56, 68, 109, 103, 77,
        24, 35, 55, 64, 81, 104, 113, 92,
        49, 64, 78, 87, 103, 121, 120, 101,
        72, 92, 95, 98, 112, 100, 103, 99
    ]).reshape(8,8)
    for n in range(num):
        for i in range(b):
            for j in range(b):
                x[n, i*8:i*8+8, j*8:j*8+8] = \
                dct2D(x[n, i*8:i*8+8, j*8:j*8+8]) / q_matrix
    y = deepcopy(x)
    x = np.round(x)
    return x, y

def restore_dct_co(x):
    """
    takes DCT coefficients as input
    generate the corresponding jpeg values
    :param x: DCT coefficients of shape(n, 128, 128)
    :return: x: jpeg pixel generated from the input DCT coefficients
    """
    num = int(x.shape[0])
    b = int(x.shape[1] / 8)
    y = deepcopy(x)
    q_matrix = np.array([
        16, 11, 10, 16, 24, 40, 51, 61,
        12, 12, 14, 19, 26, 58, 60, 55,
        14, 13, 16, 24, 40, 57, 69, 56,
        14, 17, 22, 29, 51, 87, 80, 62,
        18, 22, 37, 56, 68, 109, 103, 77,
        24, 35, 55, 64, 81, 104, 113, 92,
        49, 64, 78, 87, 103, 121, 120, 101,
        72, 92, 95, 98, 112, 100, 103, 99
    ]).reshape(8,8)
    for n in range(num):
        for i in range(b):
            for j in range(b):
                x[n, i*8:i*8+8, j*8:j*8+8] = \
                idct2D(x[n, i*8:i*8+8, j*8:j*8+8] * q_matrix)
    x = x + 128
    x[x>255] = 255; x[x<0] = 0; x = np.array(x, dtype='uint8')
    return x


In [ ]:
# generate training data 
training_data = block_training_data() 
train_k2k_x, train_k2k_y = get_dct_co(training_data)
np.save('./train/train_k2k_x.npy', train_k2k_x)
np.save('./train/train_k2k_y.npy', train_k2k_y)

In [ ]:
# validate the training data
if VALIDATION:
    train_k2k_x = np.load('./train/train_k2k_x.npy')
    train_k2k_y = np.load('./train/train_k2k_y.npy')

    restore_x = restore_dct_co(train_k2k_x)
    restore_y = restore_dct_co(train_k2k_y)

    print(train_k2k_x.shape)
    print(train_k2k_y.shape)

    for block_id in range(0, train_k2k_x.shape[0], 100):
        block_temp_x = train_k2k_x[block_id, :, :]
        implot = plt.imshow(block_temp_x)
        plt.title('train_k2k_x block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

        block_temp_y = train_k2k_y[block_id, :, :]
        implot = plt.imshow(block_temp_y)
        plt.title('train_k2k_y block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

    for block_id in range(0, restore_x.shape[0], 100):
        block_temp_x = restore_x[block_id, :, :]
        implot = plt.imshow(block_temp_x)
        plt.title('restore_x block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

        block_temp_y = restore_y[block_id, :, :]
        implot = plt.imshow(block_temp_y)
        plt.title('restore_y block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

In [ ]:
# generate testing data
test_data = block_test_data() 
test_k2k_x, test_k2k_y = get_dct_co(test_data)
np.save('test/test_k2k_x.npy', test_k2k_x)
np.save('test/test_k2k_y.npy', test_k2k_y)

In [ ]:
# validate the testing data
if VALIDATION:
    test_k2k_x = np.load('./test/test_k2k_x.npy')
    test_k2k_y = np.load('./test/test_k2k_y.npy')

    restore_x = restore_dct_co(test_k2k_x)
    restore_y = restore_dct_co(test_k2k_y)

    print(test_k2k_x.shape)
    print(test_k2k_y.shape)

    for block_id in range(0, test_k2k_x.shape[0], 20):
        block_temp_x = test_k2k_x[block_id, :, :]
        implot = plt.imshow(block_temp_x)
        plt.title('test_k2k_x block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

        block_temp_y = test_k2k_y[block_id, :, :]
        implot = plt.imshow(block_temp_y)
        plt.title('test_k2k_y block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

    for block_id in range(0, restore_x.shape[0], 20):
        block_temp_x = restore_x[block_id, :, :]
        implot = plt.imshow(block_temp_x)
        plt.title('restore_x block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()

        block_temp_y = restore_y[block_id, :, :]
        implot = plt.imshow(block_temp_y)
        plt.title('restore_y block_id = ' + str(block_id))
        plt.colorbar()
        plt.show()